In [15]:
%load_ext sql
%sql postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
import timeit
from sqlalchemy import create_engine
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Use a SELF JOIN to find athletes that share a birthday.

In [16]:
%%sql
SELECT date_of_birth, athlete_count, name
FROM (
    SELECT
        date_of_birth,
        COUNT(*) OVER (PARTITION BY date_of_birth) AS athlete_count,
        name
    FROM athletes
    GROUP BY date_of_birth, name
    ) AS subquery
WHERE athlete_count > 1
ORDER BY date_of_birth, name
LIMIT 10;

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


date_of_birth,athlete_count,name
1964-07-05,2,CHEBANIKA Raisa
1964-07-05,2,SIM Jae Yong
1967-10-12,2,PILLE-STEPPAT Sylvia
1967-10-12,2,SCHNARNDORF Susana
1970-03-24,2,FENDT Bernhard
1970-03-24,2,POMME Mauricio
1970-04-21,2,BACHMAIER Ernst
1970-04-21,2,CHIPPINGTON Jeanette
1972-06-08,2,ANDALOUSSI Ahmed
1972-06-08,2,RODRIGUEZ Matthew


In [17]:
%%sql
SELECT alias_a.date_of_birth, COUNT(*) AS athlete_count
FROM athletes alias_a INNER JOIN athletes alias_b ON alias_a.name = alias_b.name AND alias_a.date_of_birth = alias_b.date_of_birth
GROUP BY alias_a.date_of_birth
HAVING COUNT(*) > 1
LIMIT 10;


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


date_of_birth,athlete_count
1964-07-05,2
1966-10-21,4
1967-10-12,2
1968-01-23,4
1968-11-12,4
1970-03-24,2
1970-04-21,2
1970-12-26,4
1971-06-23,4
1972-06-08,2


In [18]:
%%sql 
SELECT alias_a.name athletes, alias_a.date_of_birth, alias_b.name athelete_two, alias_b.date_of_birth
FROM athletes alias_a INNER JOIN athletes alias_b
ON alias_a.date_of_birth =  alias_b.date_of_birth
WHERE alias_a.name != alias_b.name
ORDER BY alias_a.date_of_birth DESC
LIMIT 10

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


athletes,date_of_birth,athelete_two,date_of_birth_1
LEBRUN Tatyana,2004-12-09,HUANG Wenjuan,2004-12-09
HUANG Wenjuan,2004-12-09,LEBRUN Tatyana,2004-12-09
FOLGADO GARCIA Nagore,2004-03-23,CHALLIS Ellie,2004-03-23
CHALLIS Ellie,2004-03-23,FOLGADO GARCIA Nagore,2004-03-23
FELIX BARBOSA da SILVA Jardenia,2003-09-09,HONTAR Anna,2003-09-09
HONTAR Anna,2003-09-09,FELIX BARBOSA da SILVA Jardenia,2003-09-09
PEARSE Col,2003-07-10,CORSO Liza,2003-07-10
CORSO Liza,2003-07-10,PEARSE Col,2003-07-10
de LIMA Lara Aparecida,2003-04-25,TOPF Josia Tim Alexander,2003-04-25
TOPF Josia Tim Alexander,2003-04-25,de LIMA Lara Aparecida,2003-04-25


In [19]:
%%sql
SELECT alias_a.date_of_birth, COUNT(*) AS athlete_count, STRING_AGG(alias_a.name, ', ') AS athlete_names
FROM athletes alias_a INNER JOIN athletes alias_b ON alias_a.name = alias_b.name AND alias_a.date_of_birth = alias_b.date_of_birth
GROUP BY alias_a.date_of_birth
HAVING COUNT(*) > 1
LIMIT 10;

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
10 rows affected.


date_of_birth,athlete_count,athlete_names
1964-07-05,2,"SIM Jae Yong, CHEBANIKA Raisa"
1966-10-21,4,"GARRIDO MARQUEZ Victor Hugo, GARRIDO MARQUEZ Victor Hugo, GARRIDO MARQUEZ Victor Hugo, GARRIDO MARQUEZ Victor Hugo"
1967-10-12,2,"PILLE-STEPPAT Sylvia, SCHNARNDORF Susana"
1968-01-23,4,"TEUBER Michael, TEUBER Michael, TEUBER Michael, TEUBER Michael"
1968-11-12,4,"BERENYI Joseph, BERENYI Joseph, BERENYI Joseph, BERENYI Joseph"
1970-03-24,2,"FENDT Bernhard, POMME Mauricio"
1970-04-21,2,"CHIPPINGTON Jeanette, BACHMAIER Ernst"
1970-12-26,4,"SUGIURA Keiko, SUGIURA Keiko, SUGIURA Keiko, SUGIURA Keiko"
1971-06-23,4,"KEITH Aaron, KEITH Aaron, KEITH Aaron, KEITH Aaron"
1972-06-08,2,"RODRIGUEZ Matthew, ANDALOUSSI Ahmed"


In [20]:
%%timeit -n 1 -r 1
result = %sql SELECT date_of_birth, athlete_count, name FROM (SELECT date_of_birth, COUNT(*) OVER (PARTITION BY date_of_birth) AS athlete_count, name FROM athletes GROUP BY date_of_birth, name) AS subquery WHERE athlete_count > 1 ORDER BY date_of_birth, name

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1461 rows affected.
9.33 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [21]:
%%timeit -n 1 -r 1
result = %sql SELECT alias_a.date_of_birth, COUNT(*) AS athlete_count FROM athletes alias_a INNER JOIN athletes alias_b ON alias_a.name = alias_b.name AND alias_a.date_of_birth = alias_b.date_of_birth GROUP BY alias_a.date_of_birth HAVING COUNT(*) > 1

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
747 rows affected.
7.48 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
%%timeit -n 1 -r 1
result = %sql  SELECT alias_a.name athletes, alias_a.date_of_birth, alias_b.name athelete_two, alias_b.date_of_birth FROM athletes alias_a INNER JOIN athletes alias_b ON alias_a.date_of_birth =  alias_b.date_of_birth WHERE alias_a.name != alias_b.name ORDER BY alias_a.date_of_birth DESC

 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
1888 rows affected.
7.9 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [23]:
%%timeit -n 1 -r 1
result = %sql SELECT alias_a.date_of_birth, COUNT(*) AS athlete_count, STRING_AGG(alias_a.name, ', ') AS athlete_names FROM athletes alias_a INNER JOIN athletes alias_b ON alias_a.name = alias_b.name AND alias_a.date_of_birth = alias_b.date_of_birth GROUP BY alias_a.date_of_birth HAVING COUNT(*) > 1


 * postgresql+psycopg2://localhost:5432/intro_to_sql_for_analytics
747 rows affected.
10.5 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
